# Nifty Behavior
When nifty opens too high or too low, does it maintain the direction throughout the day and for how long?
- How often nifty moves up and how often it moves down?
- In market shock, does market behave in a certain way?

In [3]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *
import pytz
from logger_settings import logger

TEST_START = dt.datetime.strptime("2015-02-01", "%Y-%m-%d")
TEST_START = dt.datetime.strptime("2022-01-22", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2024-02-29", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
SYMBOL = "NIFTY MID SELECT"
IC_SYMBOL = "NIFTY"
IC_SYMBOL = "MIDCPNIFTY"
INTERVAL = INTERVAL_MIN1
EXCHANGE = EXCHANGE_NSE
pickle_file_name = f"{__name__}-2024_02_17.pkl"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = all_dates
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

def get_symbol_open(symbol, trade_dt, trade_tm, ohlc_type):
    data = ut.get_data(symbol=SYMBOL, date=trade_dt.date(), interval=INTERVAL, exchange=EXCHANGE)
    match = data.loc[data.index.time==trade_tm]
    if match.empty:
        return pd.NA
    if ohlc_type == "open":
        return data.loc[data.index.time==trade_tm].iloc[0].open
    elif ohlc_type == "close":
        return data.loc[data.index.time==trade_tm].iloc[0].close
    else:
        raise Exception(f"invalid ohlc_type {ohlc_type}")

def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

def get_symbol_first_candle_open(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open

def get_symbol_last_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[-1].close

market_open_tm = dt.time(hour=9, minute=15)
market_close_tm = dt.time(hour=15, minute=29)
train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["td_0916_open"] = train_dates.apply(lambda row: get_symbol_open(symbol=SYMBOL, trade_dt=row.name, trade_tm=market_open_tm, ohlc_type="open"), axis=1)
train_dates["td_0329_open"] = train_dates.apply(lambda row: get_symbol_open(symbol=SYMBOL, trade_dt=row.name, trade_tm=market_close_tm, ohlc_type="close"), axis=1)
train_dates["prev_0329_open"] = train_dates.td_0329_open.shift(1)

In [4]:
train_dates

,previous_trading_day,td_0916_open,td_0329_open,prev_0329_open
trade_date,,,,
2022-01-24,2022-01-21,7548.9,7251.75,None
2022-01-25,2022-01-24,7191.4,7348.55,7251.75
2022-01-27,2022-01-25,7307.3,7358.95,7348.55
2022-01-28,2022-01-27,7420.5,7414.4,7358.95
2022-01-31,2022-01-28,7504.05,7513.5,7414.4
...,...,...,...,...
2024-01-30,2024-01-29,10743.3,10609.4,10682.3
2024-01-31,2024-01-30,10598.35,10692.8,10609.4
2024-02-01,2024-01-31,10740.9,10621.4,10692.8


In [5]:
LOT_SIZE = 50
MARGIN_TIMES = 8
train_dates["long_profit"] = LOT_SIZE * (train_dates.td_0916_open - train_dates.prev_0329_open)
train_dates["short_profit"] = LOT_SIZE * (train_dates.td_0916_open - train_dates.td_0329_open)
train_dates["long_investment"] = (LOT_SIZE * train_dates.td_0916_open) / MARGIN_TIMES
train_dates["short_investment"] = (LOT_SIZE * train_dates.td_0916_open) / MARGIN_TIMES
train_dates["total_inv"] = train_dates["long_investment"] + train_dates["short_investment"]
train_dates["total_profit"] = train_dates["long_profit"] + train_dates["short_profit"]

In [6]:
"""
Test Data
Percentage profit on investment: 0.5598233376617603, total profit: 955259.9999999993, inv: 170635973.125
Percentage profit on long investment: 0.942389210522457, total profit: 804027.4999999997, inv: 85317986.5625
Percentage profit on short investment: 0.17243726197432716, total profit: 147119.99999999936, inv: 85317986.5625
Mean profit per trade: 875.5820348304302
Mean profit per long trade: 736.9637946837761
Mean profit per long trade: 134.23357664233518
"""
print(f"Percentage profit on investment: {train_dates.total_profit.sum() * 100 / train_dates.total_inv.sum()}, total profit: {train_dates.total_profit.sum()}, inv: {train_dates.total_inv.sum()}")
print(f"Percentage profit on long investment: {train_dates.long_profit.sum() * 100 / train_dates.long_investment.sum()}, total profit: {train_dates.long_profit.sum()}, inv: {train_dates.long_investment.sum()}")
print(f"Percentage profit on short investment: {train_dates.short_profit.sum() * 100 / train_dates.short_investment.sum()}, total profit: {train_dates.short_profit.sum()}, inv: {train_dates.short_investment.sum()}")
print(f"Mean profit per trade: {train_dates.total_profit.mean()}")
print(f"Mean profit per long trade: {train_dates.long_profit.mean()}")
print(f"Mean profit per long trade: {train_dates.short_profit.mean()}")

Percentage profit on investment: 1.197402264419806, total profit: 584640.0000000005, inv: 48825696.875
Percentage profit on long investment: 1.5591687343428229, total profit: 380637.50000000035, inv: 24412848.4375
Percentage profit on short investment: 0.9105451195877081, total profit: 222290.0000000003, inv: 24412848.4375
Mean profit per trade: 1173.9759036144587
Mean profit per long trade: 762.8006012024055
Mean profit per long trade: 444.5800000000006


In [21]:
train_dates

,previous_trading_day,td_0916_open,td_0329_open,prev_0329_open,long_profit,short_profit,long_investment,short_investment,total_inv,total_profit,drawdown,long_drawdown,short_drawdown
trade_date,,,,,,,,,,,,,
2015-02-04,2015-02-03,8787.15,8707.35,None,NaN,3990.0,54919.69,54919.69,109839.38,NaN,NaN,NaN,3990.0
2015-02-10,2015-02-09,8480.85,8571.05,8707.35,-11325.0,-4510.0,53005.31,53005.31,106010.62,-15835.0,-15835.0,-11325.0,-4510.0
2015-02-16,2015-02-13,8831.35,8806.7,8571.05,13015.0,1232.5,55195.94,55195.94,110391.88,14247.5,14247.5,13015.0,1232.5
2015-02-20,2015-02-19,8883.65,8830.25,8806.7,3847.5,2670.0,55522.81,55522.81,111045.62,6517.5,6517.5,3847.5,2670.0
2015-02-24,2015-02-23,8748.2,8758.9,8830.25,-4102.5,-535.0,54676.25,54676.25,109352.5,-4637.5,-4637.5,-4102.5,-535.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-20,2023-12-19,21550.55,21095.15,21410.95,6980.0,22770.0,134690.94,134690.94,269381.88,29750.0,29750.0,6980.0,22770.0
2023-12-21,2023-12-20,21023.65,21270.9,21095.15,-3575.0,-12362.5,131397.81,131397.81,262795.62,-15937.5,-15937.5,-3575.0,-12362.5
2023-12-22,2023-12-21,21272.05,21333.4,21270.9,57.5,-3067.5,132950.31,132950.31,265900.62,-3010.0,-3010.0,57.5,-3067.5


In [7]:
print(train_dates.shape[0])
train_dates = train_dates.loc[train_dates.prev_0329_open.notna() & train_dates.prev_0329_open.notnull() & train_dates.td_0916_open.notna() & train_dates.td_0329_open.notna()].copy()
print(train_dates.shape[0])
train_dates["drawdown"] = train_dates.total_profit
train_dates["long_drawdown"] = train_dates.long_profit
train_dates["short_drawdown"] = train_dates.short_profit

for i, row in train_dates.iterrows():
    prev_row = train_dates.shift(1).loc[i]
    if not prev_row.empty and not pd.isnull(prev_row.td_0329_open):
        train_dates.at[i, "drawdown"] = prev_row.drawdown + row["total_profit"]
        train_dates.at[i, "long_drawdown"] = prev_row.long_drawdown + row["long_profit"]
        train_dates.at[i, "short_drawdown"] = prev_row.short_drawdown + row["short_profit"]


502
498


In [5]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

train_dates

,previous_trading_day,td_0916_open,td_0329_open,prev_0329_open,long_profit,short_profit,long_investment,short_investment,total_inv,total_profit
trade_date,,,,,,,,,,
2021-01-01,2020-12-31,14014.85,14015.0,None,NaN,-7.5,87592.81,87592.81,175185.62,NaN
2021-01-04,2021-01-01,14097.55,14142.45,14015.0,4127.5,-2245.0,88109.69,88109.69,176219.38,1882.5
2021-01-05,2021-01-04,14071.0,14198.65,14142.45,-3572.5,-6382.5,87943.75,87943.75,175887.5,-9955.0
2021-01-06,2021-01-05,14220.45,14127.7,14198.65,1090.0,4637.5,88877.81,88877.81,177755.62,5727.5
2021-01-07,2021-01-06,14231.6,14138.1,14127.7,5195.0,4675.0,88947.5,88947.5,177895.0,9870.0
2021-01-08,2021-01-07,14237.95,14347.95,14138.1,4992.5,-5500.0,88987.19,88987.19,177974.38,-507.5
2021-01-11,2021-01-08,14431.8,14486.55,14347.95,4192.5,-2737.5,90198.75,90198.75,180397.5,1455.0
2021-01-12,2021-01-11,14458.4,14570.55,14486.55,-1407.5,-5607.5,90365.0,90365.0,180730.0,-7015.0
2021-01-13,2021-01-12,14630.0,14556.0,14570.55,2972.5,3700.0,91437.5,91437.5,182875.0,6672.5


In [8]:
"""
Test data
-52107.49999999911
-50729.99999999956
-10684.999999999536
"""
print(train_dates.drawdown.idxmin())
print(train_dates.drawdown.min())
print(train_dates.long_drawdown.min())
print(train_dates.long_drawdown.idxmin())
print(train_dates.short_drawdown.min())
print(train_dates.short_drawdown.idxmin())


2022-01-27 00:00:00
-15520.000000000027
-13409.999999999854
2022-03-08 00:00:00
-14029.999999999973
2022-02-02 00:00:00


In [9]:
train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf.total_profit.sum(), 2)}, long profit: {round(mdf.long_profit.sum(), 2)}, short_profit: {round(mdf.short_profit.sum(), 2)}")


gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf.total_profit.sum(), 2)}, long profit: {round(mdf.long_profit.sum(), 2)}, short_profit: {round(mdf.short_profit.sum(), 2)}")


Month: 2022-01-31 00:00:00, profit: -8127.5, long profit: 2480.0, short_profit: -10607.5
Month: 2022-02-28 00:00:00, profit: 12817.5, long profit: -6047.5, short_profit: 18865.0
Month: 2022-03-31 00:00:00, profit: 41157.5, long profit: 27597.5, short_profit: 13560.0
Month: 2022-04-30 00:00:00, profit: 33830.0, long profit: 15377.5, short_profit: 18452.5
Month: 2022-05-31 00:00:00, profit: 31205.0, long profit: 5962.5, short_profit: 25242.5
Month: 2022-06-30 00:00:00, profit: 15987.5, long profit: -4447.5, short_profit: 20435.0
Month: 2022-07-31 00:00:00, profit: 7002.5, long profit: 20872.5, short_profit: -13870.0
Month: 2022-08-31 00:00:00, profit: 1120.0, long profit: 9567.5, short_profit: -8447.5
Month: 2022-09-30 00:00:00, profit: 34517.5, long profit: 12345.0, short_profit: 22172.5
Month: 2022-10-31 00:00:00, profit: 33512.5, long profit: 20975.0, short_profit: 12537.5
Month: 2022-11-30 00:00:00, profit: 26795.0, long profit: 13037.5, short_profit: 13757.5
Month: 2022-12-31 00:00:

In [11]:
import historical_data as hd
x = hd.KiteUtil(exchange=EXCHANGE_NFO)
instrument = x.get_ft_instrument(symbol=IC_SYMBOL, expiry=dt.datetime.strptime("2024-04-25", "%Y-%m-%d").date())
instrument = x.get_ft_instrument(symbol=IC_SYMBOL, expiry=dt.datetime.strptime("2024-03-22", "%Y-%m-%d").date())
start = dt.datetime.strptime("2020-01-01", "%Y-%m-%d").date()
end = dt.datetime.now().date()
# x.fetch_stock_data_it(instrument_token=9372674, from_date=start, to_date: datetime, interval, continuous=True)
nifty_fd = x.kite.historical_data(instrument_token=instrument["instrument_token"], from_date=start, to_date=end, interval=INTERVAL_DAY, continuous=True)
nifty_fd = pd.DataFrame(nifty_fd)
nifty_fd.set_index('date', inplace=True)

In [20]:
train_dates

,previous_trading_day,td_0916_open,td_0329_open,prev_0329_open,long_profit,short_profit,long_investment,short_investment,total_inv,total_profit,drawdown,long_drawdown,short_drawdown,trade_date
trade_date,,,,,,,,,,,,,,
2021-01-04,2021-01-01,14097.55,14142.45,14015.0,4127.5,-2245.0,88109.69,88109.69,176219.38,1882.5,1882.5,4127.5,-2245.0,2021-01-04
2021-01-05,2021-01-04,14071.0,14198.65,14142.45,-3572.5,-6382.5,87943.75,87943.75,175887.5,-9955.0,-8072.5,555.0,-8627.5,2021-01-05
2021-01-06,2021-01-05,14220.45,14127.7,14198.65,1090.0,4637.5,88877.81,88877.81,177755.62,5727.5,-2345.0,1645.0,-3990.0,2021-01-06
2021-01-07,2021-01-06,14231.6,14138.1,14127.7,5195.0,4675.0,88947.5,88947.5,177895.0,9870.0,7525.0,6840.0,685.0,2021-01-07
2021-01-08,2021-01-07,14237.95,14347.95,14138.1,4992.5,-5500.0,88987.19,88987.19,177974.38,-507.5,7017.5,11832.5,-4815.0,2021-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-21,2024-02-20,22202.95,22023.7,22206.45,-175.0,8962.5,138768.44,138768.44,277536.88,8787.5,789610.0,583705.0,205905.0,2024-02-21
2024-02-22,2024-02-21,22050.2,22227.8,22023.7,1325.0,-8880.0,137813.75,137813.75,275627.5,-7555.0,782055.0,585030.0,197025.0,2024-02-22
2024-02-23,2024-02-22,22249.1,22195.85,22227.8,1065.0,2662.5,139056.88,139056.88,278113.75,3727.5,785782.5,586095.0,199687.5,2024-02-23


In [18]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)
after_jan = train_dates
# after_jan = train_dates[train_dates.index.date > dt.datetime(year=2022, month=1, day=1).date()].copy()
def get_fut_day_open(date, tm):
    print(date)
    # print(tm)
    return nifty_fd[(nifty_fd.index.date == date) & (nifty_fd.index.time == tm)].iloc[0].open

def get_fut_day(date, open):
    x = nifty_fd.loc[(nifty_fd.index.date == date)]
    if x.empty:
        print(date)
        return pd.NA
    if open:
        return nifty_fd[(nifty_fd.index.date == date)].iloc[0].open
    else:
        return nifty_fd[(nifty_fd.index.date == date)].iloc[0].close

# after_jan["fut_prev_open"] = after_jan.apply(lambda r: get_fut_day_open(r.previous_trading_day.date(), market_close_tm), axis=1)
# after_jan["fut_td_open"] = after_jan.apply(lambda r: get_fut_day_open(r.name.date(), market_open_tm), axis=1)
# after_jan["fut_td_close"] = after_jan.apply(lambda r: get_fut_day_open(r.name.date(), market_close_tm), axis=1)

after_jan["fut_prev_open"] = after_jan.apply(lambda r: get_fut_day(r.previous_trading_day.date(), False), axis=1)
after_jan["fut_td_open"] = after_jan.apply(lambda r: get_fut_day(r.name.date(), True), axis=1)
after_jan["fut_td_close"] = after_jan.apply(lambda r: get_fut_day(r.name.date(), False), axis=1)

after_jan["fut_td_diff"] = after_jan.fut_td_open - after_jan.fut_prev_open
after_jan["long_diff"] = after_jan.td_0916_open - after_jan.prev_0329_open
after_jan["fl_diff"] = after_jan.long_diff - after_jan.fut_td_diff
after_jan

2023-04-26
2023-04-26
2023-04-26


,previous_trading_day,td_0916_open,td_0329_open,prev_0329_open,long_profit,short_profit,long_investment,short_investment,total_inv,total_profit,drawdown,long_drawdown,short_drawdown,trade_date,fut_prev_open,fut_td_open,fut_td_close,fut_td_diff,long_diff,fl_diff
trade_date,,,,,,,,,,,,,,,,,,,,
2022-01-25,2022-01-24,7191.4,7348.55,7251.75,-3017.5,-7857.5,44946.25,44946.25,89892.5,-10875.0,-10875.0,-3017.5,-7857.5,2022-01-25,7254.05,7108.65,7355.3,-145.4,-60.35,85.05
2022-01-27,2022-01-25,7307.3,7358.95,7348.55,-2062.5,-2582.5,45670.62,45670.62,91341.25,-4645.0,-15520.0,-5080.0,-10440.0,2022-01-27,7355.3,7290.0,7359.35,-65.3,-41.25,24.05
2022-01-28,2022-01-27,7420.5,7414.4,7358.95,3077.5,305.0,46378.12,46378.12,92756.25,3382.5,-12137.5,-2002.5,-10135.0,2022-01-28,7359.35,7475.95,7425.3,116.6,61.55,-55.05
2022-01-31,2022-01-28,7504.05,7513.5,7414.4,4482.5,-472.5,46900.31,46900.31,93800.62,4010.0,-8127.5,2480.0,-10607.5,2022-01-31,7425.3,7528.7,7528.75,103.4,89.65,-13.75
2022-02-01,2022-01-31,7586.7,7605.9,7513.5,3660.0,-960.0,47416.88,47416.88,94833.75,2700.0,-5427.5,6140.0,-11567.5,2022-02-01,7528.75,7582.0,7610.85,53.25,73.2,19.95
2022-02-02,2022-02-01,7640.7,7689.95,7605.9,1740.0,-2462.5,47754.38,47754.38,95508.75,-722.5,-6150.0,7880.0,-14030.0,2022-02-02,7610.85,7610.0,7700.0,-0.85,34.8,35.65
2022-02-03,2022-02-02,7677.3,7604.7,7689.95,-632.5,3630.0,47983.12,47983.12,95966.25,2997.5,-3152.5,7247.5,-10400.0,2022-02-03,7700.0,7665.05,7617.55,-34.95,-12.65,22.3
2022-02-04,2022-02-03,7598.6,7502.4,7604.7,-305.0,4810.0,47491.25,47491.25,94982.5,4505.0,1352.5,6942.5,-5590.0,2022-02-04,7617.55,7590.05,7511.9,-27.5,-6.1,21.4
2022-02-07,2022-02-04,7524.75,7442.75,7502.4,1117.5,4100.0,47029.69,47029.69,94059.38,5217.5,6570.0,8060.0,-1490.0,2022-02-07,7511.9,7551.35,7506.65,39.45,22.35,-17.1


In [20]:
LOT_SIZE = 50
MARGIN_TIMES = 8
after_jan = after_jan[after_jan.fut_prev_open.notna()].copy()
after_jan["flong_profit"] = LOT_SIZE * (after_jan.fut_td_open - after_jan.fut_prev_open)
after_jan["fshort_profit"] = LOT_SIZE * (after_jan.fut_td_open - after_jan.fut_td_close)
after_jan["flong_investment"] = (LOT_SIZE * after_jan.fut_prev_open) / MARGIN_TIMES
after_jan["fshort_investment"] = (LOT_SIZE * after_jan.fut_td_open) / MARGIN_TIMES
after_jan["ftotal_inv"] = after_jan["flong_investment"] + after_jan["fshort_investment"]
after_jan["ftotal_profit"] = after_jan["flong_profit"] + after_jan["fshort_profit"]

In [21]:
"""
First expiry
Month: 2024-01-31 00:00:00, profit: 30440.0, long profit: 19975.0, short_profit: 10465.0
fMonth: 2024-01-31 00:00:00, profit: 19615.0, long profit: 12812.5, short_profit: 6802.5
Month: 2024-02-29 00:00:00, profit: -14430.0, long profit: 10610.0, short_profit: -25040.0
fMonth: 2024-02-29 00:00:00, profit: -14290.0, long profit: 8907.5, short_profit: -23197.5
Year: 2024-12-31 00:00:00, profit: 16010.0, long profit: 30585.0, short_profit: -14575.0
fYear: 2024-12-31 00:00:00, profit: 5325.0, long profit: 21720.0, short_profit: -16395.0
"""

after_jan["trade_date"] = after_jan.index.values
gdf = after_jan.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf.total_profit.sum(), 2)}, long profit: {round(mdf.long_profit.sum(), 2)}, short_profit: {round(mdf.short_profit.sum(), 2)}")
    print(f"fMonth: {month}, profit: {round(mdf.ftotal_profit.sum(), 2)}, long profit: {round(mdf.flong_profit.sum(), 2)}, short_profit: {round(mdf.fshort_profit.sum(), 2)}")


gdf = after_jan.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf.total_profit.sum(), 2)}, long profit: {round(mdf.long_profit.sum(), 2)}, short_profit: {round(mdf.short_profit.sum(), 2)}")
    print(f"fYear: {year}, profit: {round(mdf.ftotal_profit.sum(), 2)}, long profit: {round(mdf.flong_profit.sum(), 2)}, short_profit: {round(mdf.fshort_profit.sum(), 2)}")

    # gdf = train_dates.resample('D')['gap'].groupby(df.index.dt.dayofweek).agg(mean='mean', sum='sum')
    dgdf = mdf.groupby(mdf.index.dayofweek)
    for month, dmdf in dgdf:
        print(f"Month: {month}, profit: {round(dmdf.total_profit.sum(), 2)}, long profit: {round(dmdf.long_profit.sum(), 2)}, short_profit: {round(dmdf.short_profit.sum(), 2)}")
        print(f"Month: {month}, profit: {round(dmdf.ftotal_profit.sum(), 2)}, long profit: {round(dmdf.flong_profit.sum(), 2)}, short_profit: {round(dmdf.fshort_profit.sum(), 2)}")


Month: 2022-01-31 00:00:00, profit: -8127.5, long profit: 2480.0, short_profit: -10607.5
fMonth: 2022-01-31 00:00:00, profit: -12805.0, long profit: 465.0, short_profit: -13270.0
Month: 2022-02-28 00:00:00, profit: 12817.5, long profit: -6047.5, short_profit: 18865.0
fMonth: 2022-02-28 00:00:00, profit: 34125.0, long profit: 4875.0, short_profit: 29250.0
Month: 2022-03-31 00:00:00, profit: 41157.5, long profit: 27597.5, short_profit: 13560.0
fMonth: 2022-03-31 00:00:00, profit: 12125.0, long profit: 13057.5, short_profit: -932.5
Month: 2022-04-30 00:00:00, profit: 33830.0, long profit: 15377.5, short_profit: 18452.5
fMonth: 2022-04-30 00:00:00, profit: 56932.5, long profit: 25612.5, short_profit: 31320.0
Month: 2022-05-31 00:00:00, profit: 31205.0, long profit: 5962.5, short_profit: 25242.5
fMonth: 2022-05-31 00:00:00, profit: 42340.0, long profit: 12547.5, short_profit: 29792.5
Month: 2022-06-30 00:00:00, profit: 15987.5, long profit: -4447.5, short_profit: 20435.0
fMonth: 2022-06-30 

In [45]:
after_jan["fl_ratio"] = after_jan["fut_td_diff"] / after_jan["long_diff"]
after_jan[after_jan.index.year == 2022][["long_diff", "fut_td_diff", "fl_diff", "fl_ratio"]]

,long_diff,fut_td_diff,fl_diff,fl_ratio
trade_date,,,,
2022-01-03,22.9,96.15,-73.25,4.2
2022-01-04,46.05,6.85,39.2,0.15
2022-01-05,12.75,41.05,-28.3,3.22
2022-01-06,-139.2,-138.80,-0.4,1.0
2022-01-07,51.6,41.95,9.65,0.81
2022-01-10,95.1,98.60,-3.5,1.04
2022-01-11,-7.7,-16.75,9.05,2.18
2022-01-12,111.4,66.80,44.6,0.6
2022-01-13,59.4,42.40,17.0,0.71


In [ ]:
print(after_jan.shape[0])
after_jan = after_jan.loc[after_jan.prev_0329_open.notna() & after_jan.prev_0329_open.notnull() & after_jan.td_0916_open.notna() & after_jan.td_0329_open.notna()].copy()
print(after_jan.shape[0])
after_jan["drawdown"] = after_jan.total_profit
after_jan["long_drawdown"] = after_jan.long_profit
after_jan["short_drawdown"] = after_jan.short_profit

for i, row in train_dates.iterrows():
    prev_row = train_dates.shift(1).loc[i]
    if not prev_row.empty and not pd.isnull(prev_row.td_0329_open):
        train_dates.at[i, "drawdown"] = prev_row.drawdown + row["total_profit"]
        train_dates.at[i, "long_drawdown"] = prev_row.long_drawdown + row["long_profit"]
        train_dates.at[i, "short_drawdown"] = prev_row.short_drawdown + row["short_profit"]
